## Understanding the problem

We have a time dynamical system whose state is described by two variables, $x$ and $v$.

The question asks how we can approximate these time dynamical systems using a regression tree when the data is in the form
$$
((x_k, v_k), (x_{k+1}, v_{k+1}))
$$

We could use a single tree to make predictions for the next state, but we would get more accurate results if we used multiple regression trees each with a limited height?

## Code for task 4

In [1]:
from task1 import *

# generating data
T = np.arange(0, 20)

# initial state
x_k = np.array([0.5, 1.5])

# creating an array of data
# shape: (number of time steps (rows), number of entries in each row, dimension of each column (previous state and next state))
X = np.zeros((T.size, 2, 2))

for t in T:
    X[t, 0, :] = x_k
    # creating next state
    X[t, 1, 0] = 0.9*x_k[0] - 0.2*x_k[1]
    X[t, 1, 1] = 0.2*x_k[0] + 0.9*x_k[1]
    x_k = X[t, 1, :]

y = X[:, 1]
X = X[:, 0]
X.shape

(20, 2)

In [2]:
print(y[0:2])
np.mean(y[0:2], axis=0)

[[ 0.15   1.45 ]
 [-0.155  1.335]]


array([-0.0025,  1.3925])

In [6]:
model = RegressionTree()
model.fit(X, y)
model.decision_path(X[10])
print(y[10])

Length of queue: 1
x[0] < 0.14999999999999997
x[0] < -0.4065000000000001
x[0] >= -0.7343850000000003
x[0] >= -0.6714284655500002
x[0] < -0.5636064712650002
x == [-0.734385  0.754945]
[-0.56360647 -0.31734186]
